In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with open('names.txt', 'r') as f:
    words = f.read().splitlines()

In [7]:
# build vocabulary
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)

In [9]:
# build dataset
block_size = 3
def build_dataset(words):
    X, Y = [], []


    for w in words:
        context = [0] * block_size

        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182580, 3]) torch.Size([182580])
torch.Size([22767, 3]) torch.Size([22767])
torch.Size([22799, 3]) torch.Size([22799])


In [44]:
# MLP
n_embd = 10
n_hidden = 200

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size,n_embd)                 , generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden)    , generator=g) * 0.2
b1 = torch.randn(n_hidden                           , generator=g) * 0.01
W2 = torch.randn((n_hidden, vocab_size)             , generator=g) * 0.01
b2 = torch.randn(vocab_size                         , generator=g) * 0

bngain = torch.ones((1, n_hidden))
bnbias = torch.ones((1, n_hidden))
bnmean_running = torch.ones((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, b1, W2, b2, bngain, bnbias]

for p in parameters:
    p.requires_grad = True

In [ ]:
max_steps = 200000
batch_size = 32
lossi = []

# train
for i in range(max_steps):
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]

    # forward pass
    emb = C[Xb]
    embdcat = emb.view(emb.shape[0], -1)
    hpreact = embdcat @ W1 #+ b1
    # batch normalization, scale and shift
    bnmeani = hpreact.mean(0, keepdim=True)
    bnstdi = hpreact.std(0, keepdim=True)
    hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias

    with torch.no_grad():
        bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
        bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi

    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Yb) #softmax + loss
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
    if i % 10000 == 0:
        print(f"{i:7d}/{max_steps:7d}: {loss.item():.4f}")
    lossi.append(loss.log10().item())

In [ ]:
plt.plot(lossi)

In [41]:
emb = C[Xtr]
embcat = emb.view(emb.shape[0], -1)
hpreact =  embcat @ W1 #+ b1
hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
h = torch.tanh(hpreact)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

tensor(2.0344, grad_fn=<NllLossBackward0>)

In [42]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 30) @ W1 #+ b1
               )
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

tensor(2.1024, grad_fn=<NllLossBackward0>)

In [43]:
# sampling
gs = torch.Generator().manual_seed(2147483647 + 10)

for i in range(10):
    out = []
    context = [0] * 3

    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 
                       #b1
                       )
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)

        ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=gs).item()
        context = context[1:] + [ix]
        out.append(ix)
        
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))

mora.
mayanniellen.
hayla.
remmadiendra.
gradeeko.
eliah.
milopileigh.
esmanar.
kate.
malaia.


In [ ]:
# fixing the initial loss
# fixing the saturated tanh
# calculating the init scale
# batch normalization